In [2]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import os

CLIENT_SECRET_FILE = 'client_secret2.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']

flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = build('youtube', 'v3', credentials=credentials)

#https://console.developers.google.com/iam-admin/quotas?project=api-project-233696733563
#https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=isentropic-now-275613


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=41469420950-bfl3nb76jittq70tjbun4eh00k403as4.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=od9imaQtarro1VaaenIhy5vyMbfFRI&prompt=consent&access_type=offline
Enter the authorization code: 4/1wE4RBRmO2eL3wzBr4JTVTAN0UpNspABQ7AchlHrBLA8-z16z4uyyyY


In [3]:
## get all playlist of channel
def get_playlist(token):
    request = youtube.playlists().list(
            part="snippet,contentDetails",
            mine=True,
            maxResults=50,
            pageToken=token
        )
    response = request.execute()
    return response
    
## get list of videos from youtube
def get_video_list(token,vtype):
    request = youtube.search().list(
        part="snippet",
        forMine=True,
        type=vtype,
        maxResults=50,
        order="date",
        pageToken=token
    )
    response = request.execute()
    #print(response)
    return response


## get playlist form youtube by playlist id 
def get_playlist_videos(playlistid, token):
    request = youtube.playlistItems().list(
            playlistId=playlistid,
            part="contentDetails,status,snippet",
            maxResults=50,
            pageToken=token
        )
    response = request.execute() 
    #f=response['items'][0]
    #print(f)
    return response

## get list of video from youtube channel 
def show_videolist():
    my_uploaded_video_playlist='UUC5hntNsVE7rQRYqqv_ON0g'
    data =pd.DataFrame()
    count =0
    size=50
    max_item=1
    nextPageToken =None
    while(count*size <= max_item) :
        rs=get_playlist_videos(my_uploaded_video_playlist,nextPageToken)
        #rs=get_video_list(nextPageToken,'video')
        max_item=int(rs['pageInfo']['totalResults'])
        nextPageToken= rs.get('nextPageToken')
        val=[{'id':d['contentDetails']['videoId'], 'title' :d['snippet']['title'], 'status':d['status']['privacyStatus'] }  for d in rs['items']] 
        print(len(val),max_item,nextPageToken)
        data=data.append(val)
        count+=1  
    return data

## get list of playlist avaliable in channel
def show_youtube_playlist():
    data =pd.DataFrame()
    count =0
    size=50
    max_item=1
    nextPageToken =None
    while(count*size <= max_item) :
        rs=get_playlist(nextPageToken)
        max_item=int(rs['pageInfo']['totalResults'])
        #print(rs)
        nextPageToken= rs.get('nextPageToken')
        val=[{ 'playlist_id': item['id'], 'playlist_title': item['snippet']['title'] ,'video_count': item["contentDetails"]["itemCount"]} for item in rs['items']]
        data = data.append(val)
        count+=1  
    return data


## creating new playlist in youtube 
def create_playlist(title):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
          "snippet": {
            "title":  title,
            "description":  title,
            "tags": [
              "ML,AI "          
            ],
            "defaultLanguage": "en"
          },
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response is not None and 'id' in response) :
        print("playlist added successfully with id :{0}".format(response["id"]))
    else:
          print("failed to create playlist ")
          print(response)

## add video in youtube playlist by playlist id , video id
def add_playlist_item(playlist_id, video_id):
    request = youtube.playlistItems().insert(
        part="snippet",
        body={
          "snippet": {
            "playlistId": playlist_id,
            "resourceId": {
              "kind": "youtube#video",
              "videoId": video_id
            }
          }
        }
    )
    response = request.execute()
    if(response.get('id') is not None) :
        print('item added ' + str(id))
    #print(response)

def UpdateVideo(id):
    request = youtube.videos().update(
        part="status",
        body={
          "id": id,
          "status": {
            "privacyStatus": "Unlisted"
          }
        }
    )
    response = request.execute()
    if(response.get('id') is not None):
        print('video updated - ' +response.get('id'))
    else :
        print(response)    
    
    
##delete playlist by id from youtube 
def delete_playlist(playlist_id):
     request =  youtube.playlists().delete(id=playlist_id)
     response = request.execute()
     print(response)

        
## get list of videos from physical location of videos avaliable in folder structure         
def get_video_files(fullpath):
    filter_extension=".ts"
    data = [ { 'course_type' :path.split('\\')[6].split('(')[0][-1],
               'playlist' :"ML & AI |" + '|'.join(path.split('\\')[6:]) , \
               'file_name':os.path.splitext(file)[0],  "path" : path} \
            for path, subdirs, files in os.walk(fullpath) for file in files if file.endswith(filter_extension)] 
    df= pd.DataFrame(data)
    return df


## create playlist from physcial location of folder structure
def create_playlist_data(fullpath, ptype=None):
    df =get_video_files(fullpath)
    if(ptype is not None) :
        df = df[df['course_type']== str(ptype)]     
    groups=df.groupby('playlist')
    return list(groups.groups.keys())



In [ ]:

#add videolist item
#add_playlist_item(youtube,"PL9tQy71sPLeHoB-g1H08J1hR8ZsXWzz7Z","05yT5rR-XsM")



#get_list(None,'video')
#create playlist

#create_playlist(youtube,"")

#delete_playlist()




In [4]:
##list of videos from youtube 
rs= show_videolist()
rs.to_csv('youtube_videlist.csv', index=False)
len(rs)

50 1118 CDIQAA
50 1118 CGQQAA
50 1118 CJYBEAA
50 1118 CMgBEAA
50 1118 CPoBEAA
50 1118 CKwCEAA
50 1118 CN4CEAA
50 1118 CJADEAA
50 1118 CMIDEAA
50 1118 CPQDEAA
50 1118 CKYEEAA
50 1118 CNgEEAA
50 1118 CIoFEAA
50 1118 CLwFEAA
50 1118 CO4FEAA
50 1118 CKAGEAA
50 1118 CNIGEAA
50 1118 CIQHEAA
50 1118 CLYHEAA
50 1118 COgHEAA
50 1118 CJoIEAA
50 1118 CMwIEAA
18 1118 None


1118

In [11]:
##create video list from dir path
df=get_video_files(r"C:\Users\Ashwini\Downloads\PGDM\Video")
df.to_csv('dir_file_status.csv', index=False)


In [12]:
##show playlists from youtube
df=show_youtube_playlist()
df.to_csv('youtube_playlist.csv', index=False)

In [6]:
# create playlist by this on youtube 

dt=create_playlist_data(r"C:\Users\Ashwini\Downloads\PGDM\Video",5)

print(len(dt))

#resume from 10
for g in dt[14:]: #remaining
    print(g)
    #create_playlist(g)




20
ML & AI |Course 5(AI-2)|M-4(Recurrent Neural Networks-DL)|S-1(What Makes a Neural Network Recurrent)
playlist added successfully with id :PLMCuzyPpc7o4b8NljjJOGmA1YJRe-Czd0
ML & AI |Course 5(AI-2)|M-4(Recurrent Neural Networks-DL)|S-2(Variants of RNNs)
playlist added successfully with id :PLMCuzyPpc7o6shhgj2-gib26MYY8EQG4A
ML & AI |Course 5(AI-2)|M-4(Recurrent Neural Networks-DL)|S-3(Building RNNs in Python)
playlist added successfully with id :PLMCuzyPpc7o7GgXvmyxlUI4lryI8vIyJn
ML & AI |Course 5(AI-2)|M-4(Recurrent Neural Networks-DL)|S-4(1D Convolutions)
playlist added successfully with id :PLMCuzyPpc7o7sd0SBjeg0ZlMk8TL8XklX
ML & AI |Course 5(AI-2)|M-4(Recurrent Neural Networks-DL)|S-5(Additional Resources)
playlist added successfully with id :PLMCuzyPpc7o75QWN3dLNkNHQfSLJNRV7e
ML & AI |Course 5(AI-2)|M-5(Neural Networks Project Gesture Recognition-DL)
playlist added successfully with id :PLMCuzyPpc7o7pfXj-feIRVU4vwVIsO8hi


In [5]:
##get file to map youtube video with playlist

file_status = pd.read_csv('dir_file_status.csv')
youtube_videlist = pd.read_csv('youtube_videlist.csv')
youtube_playlist= pd.read_csv('youtube_playlist.csv')


del file_status['path']
file_status['file_name']= file_status['file_name'] \
.str.replace('-',' ').str.replace('(','').str.replace(')','').str.replace('.',' ')


print(len(file_status['file_name']),len(youtube_videlist['title']))

rs1=file_status.merge(youtube_videlist,left_on='file_name', right_on='title', how='left')
rs2=rs1.merge(youtube_playlist,left_on='playlist', right_on='playlist_title', how='left')
rs2.to_csv('result.csv', index=False)

#file_status['file_name']==youtube_videlist['title']

#rs1[rs1['id'].isnull()].to_csv('not_uploaded.csv')


281 1118


In [42]:
##LINK VIDEO WITH PLAYLIST

rs = pd.read_csv('result.csv')

rs=rs[rs['course_type']==5]
rs= rs[ (rs['video_count']==0) & (~rs['id'].isnull()) & (~rs['playlist_id'].isnull()) ] # and  != ""

print(len(rs))

rs.sort_values(by=['title'], inplace=True)

rs= rs[:]

#for i,r in rs.iterrows(): #116
#    print(i,r['playlist_id'],r['id'],r['title'])
    #add_playlist_item(r['playlist_id'],r['id'])
    
    
#rs.isnull().sum()
# 201 total

100
0


In [7]:
#update youtube video access level
rs = pd.read_csv('youtube_videlist.csv')

rs=rs[rs['status']=='private']
print(len(rs))

rs= rs[:]

for i,r in rs.iterrows(): #116
    print(i,r['id'])
    UpdateVideo(r['id']) 
    
#last 674   KHBJeAhUnU0

50
0 ArpBRpAHXbg
video updated - ArpBRpAHXbg
1 SMGEmGjs4Zo
video updated - SMGEmGjs4Zo
2 FknK92a6Hac
video updated - FknK92a6Hac
3 8eEGTCetElo
video updated - 8eEGTCetElo
4 Qv8pD3x5XeU
video updated - Qv8pD3x5XeU
5 YjLONljXAYI
video updated - YjLONljXAYI
6 a1L3n2dN3D4
video updated - a1L3n2dN3D4
7 nTg3TyDPd0Y
video updated - nTg3TyDPd0Y
8 _DSqts9Li1k
video updated - _DSqts9Li1k
9 WDeXCfsrSQo
video updated - WDeXCfsrSQo
10 53sGteKuToY
video updated - 53sGteKuToY
11 9ovXJtNWeVg
video updated - 9ovXJtNWeVg
12 vJ0juE_QbqE
video updated - vJ0juE_QbqE
13 VKLjgQMevTE
video updated - VKLjgQMevTE
14 uUlHWQO3lDc
video updated - uUlHWQO3lDc
15 DmABKM6pMuE
video updated - DmABKM6pMuE
16 EysLY-hF7IA
video updated - EysLY-hF7IA
17 yOk7y_7_U0A
video updated - yOk7y_7_U0A
18 WwuHmzbuDGc
video updated - WwuHmzbuDGc
19 WsjmDVYnt-k
video updated - WsjmDVYnt-k
20 tXEh_9d-Oac
video updated - tXEh_9d-Oac
21 a4zMx8sALw4
video updated - a4zMx8sALw4
22 T1nSvdAF-uU
video updated - T1nSvdAF-uU
23 Os5COwMcoVk
vid

In [143]:
## create html file to structure course videos list

import pandas as pd

list_data = pd.read_csv('youtube_playlist.csv')

dt=pd.DataFrame.from_records(list_data['playlist_title'].str.split('|'),columns=['Package' ,'Course','Module','Session'])

list_data=pd.concat([list_data,dt], axis=1)

list_data.sort_values(by=['playlist_title'], inplace=True)

def create_html_list(source_df):
    data="<div style='font-family:    Arial, Helvetica, sans-serif'>"
    link="<ul><a target='_blank' href='https://www.youtube.com/playlist?list={1}'>{0}</a></ul>"

    for g1, data1 in source_df.groupby(['Package']):
        data+= "<ul>" + g1 
        #print(g1)
        for g2, data2 in data1.groupby(['Course']):
            data+= "<ul>" + g2
            #print(g2)
            for g3, data3 in data2.groupby(['Module']):      
                if(len(data3.groupby(['Session']).groups)==0):
                    data+=link.format(g3,data3.iloc[0]['playlist_id']) 
                else :
                    data+= "<ul>" + g3
                    for g4, data4 in data3.groupby(['Session']):
                         data+=link.format(g4,data4.iloc[0]['playlist_id'])                   
                    data+="</ul>"  
            data+="</ul>"  
        data+="</ul>"       

    data+="</div>"

    text_file = open("index.html", "w")
    text_file.write(data)
    text_file.close()
    
create_html_list(list_data)